In [9]:
import tensorflow as tf
import pandas as pd
import cv2
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

2022-03-16 06:36:50.080513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:50.081403: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:50.087717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:50.088504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:50.089200: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [10]:
nyu_df = pd.read_csv('./data/nyu2_train.csv', header=None)
nyu_df.columns = ['rgb', 'depth']
nyu_df

,rgb,depth
0,data/nyu2_train/living_room_0038_out/37.jpg,data/nyu2_train/living_room_0038_out/37.png
1,data/nyu2_train/living_room_0038_out/115.jpg,data/nyu2_train/living_room_0038_out/115.png
2,data/nyu2_train/living_room_0038_out/6.jpg,data/nyu2_train/living_room_0038_out/6.png
3,data/nyu2_train/living_room_0038_out/49.jpg,data/nyu2_train/living_room_0038_out/49.png
4,data/nyu2_train/living_room_0038_out/152.jpg,data/nyu2_train/living_room_0038_out/152.png
...,...,...
50683,data/nyu2_train/dining_room_0031_out/441.jpg,data/nyu2_train/dining_room_0031_out/441.png
50684,data/nyu2_train/dining_room_0031_out/365.jpg,data/nyu2_train/dining_room_0031_out/365.png
50685,data/nyu2_train/dining_room_0031_out/447.jpg,data/nyu2_train/dining_room_0031_out/447.png
50686,data/nyu2_train/dining_room_0031_out/90.jpg,data/nyu2_train/dining_room_0031_out/90.png


In [11]:
img = tf.io.decode_image(tf.io.read_file(nyu_df['depth'][0]))

2022-03-16 06:36:59.174774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-16 06:36:59.298310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:59.299119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 06:36:59.299782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [12]:
dot_pattern_ = cv2.imread("./data/kinect-pattern_3x3.png", 0)

focal_length  = 582.7  # focal length of the camera used 
baseline_m    = 0.075
range_max=10;
range_min=0;
kernel_size= np.random.randint(1,45)
blurr_angle= np.random.randint(1,45)


libpng warning: iCCP: known incorrect sRGB profile


In [13]:
from utils.make_noisy_depth.make_noisy_image import *

In [14]:
def make_noisy_data(depth_path):
    scale_factor  = 100
    invalid_disp_ = 99999999.9
    
    img1 = tf.io.decode_image(tf.io.read_file(depth_path))
    img1 = np.squeeze(img1.numpy())
    
    h, w = img1.shape[0], img1.shape[1]

    depth =  img1.astype('float')/255.0*10

    depth_interp = add_gaussian_shifts(depth, focal_length,baseline_m)

    disp_= focal_length * baseline_m / (depth_interp + 1e-10)

    depth_f = np.round(disp_ * 8.0)/8.0

    out_disp = filterDisp(depth_f, dot_pattern_, invalid_disp_)

    depth = focal_length * baseline_m / out_disp

    depth[out_disp == invalid_disp_] = 0

    noisy_depth = make_axial_noise(depth,range_max,range_min)
    noisy_depth = noisy_depth * 255.0/10
    noisy_depth = noisy_depth.astype('uint8')
    
    #blurr_image1=make_blurr(noisy_depth,kernel_size,blurr_angle)
    
    
    return noisy_depth

In [15]:
nyu_df['depth']

0         data/nyu2_train/living_room_0038_out/37.png
1        data/nyu2_train/living_room_0038_out/115.png
2          data/nyu2_train/living_room_0038_out/6.png
3         data/nyu2_train/living_room_0038_out/49.png
4        data/nyu2_train/living_room_0038_out/152.png
                             ...                     
50683    data/nyu2_train/dining_room_0031_out/441.png
50684    data/nyu2_train/dining_room_0031_out/365.png
50685    data/nyu2_train/dining_room_0031_out/447.png
50686     data/nyu2_train/dining_room_0031_out/90.png
50687    data/nyu2_train/dining_room_0031_out/564.png
Name: depth, Length: 50688, dtype: object

In [16]:
def make_train_ds(depth_path):
    noisy_depth = make_noisy_data(depth_path)
    #Save    
    save_path = './data/noisy_train/'+depth_path[16:-3] + 'jpg'
#     print(save_path)
    cv2.imwrite(save_path, noisy_depth)
    return save_path

In [17]:
pbar = tqdm(nyu_df['depth'])
for depth_path in pbar:
    save_path = make_train_ds(depth_path)
    pbar.set_description(f"saving data:{save_path}")


saving data:./data/noisy_train/kitchen_0035b_out/39.jpg:  85%|██████████████████████████████████████████████████████████████████████████████████████████▊                | 43026/50688 [68:12:02<34:54:29, 16.40s/it]IOStream.flush timed out
saving data:./data/noisy_train/dining_room_0031_out/564.jpg: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 50688/50688 [80:07:48<00:00,  5.69s/it]
